In [ ]:
import pandas as pd
import numpy as np

Сначала о видах признаков. И о том как их представить в "читаемом\понятном" для модели виде

# Непрерывные признаки

тут все ок.

# Категории

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

##### Label

![](https://qph.fs.quoracdn.net/main-qimg-715744b45247794c88f6b68beb744ad4)

##### One hot

![](https://i.imgur.com/mtimFxh.png)

_____

##### Тексты

![](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/assets/feml_0405.png)

в sklearn для этого есть CountVectorizer

# Пропуски

1. как есть

In [ ]:
df = pd.DataFrame({'площадь': [80, np.nan, 83, 74],
              'цена':[200, 250, 180, np.nan], 
              }) #'цена_кв_м':[200 / 80, 250 / 70, np.nan, np.nan]

In [ ]:
df

In [ ]:
df['цена_кв_м'] = df['цена'] / df['площадь']

In [ ]:
df1 = df
df

2. заполним сразу.

In [ ]:
df = pd.DataFrame({'площадь': [80, np.nan, 83, 74],
              'цена':[200, 250, 180, np.nan], 
              }) #'цена_кв_м':[200 / 80, 250 / 70, np.nan, np.nan]

In [ ]:
#TODO Заполнить пропуски средними сразу

3. заполним после генерации фичей

In [ ]:
df = pd.DataFrame({'площадь': [80, np.nan, 83, 74],
              'цена':[200, 250, 180, np.nan], 
              }) #'цена_кв_м':[200 / 80, 250 / 70, np.nan, np.nan]

In [ ]:
#TODO Заполнить пропуски средним после генерации признаков

Итого:

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

> Отсюда нужно сделать вывод что сразу лезть заполнять пропуски - плохая идея, если вы потом будете генерить фичи. Дело конечно ваше, к разным задачам разный подход, но обычно нужно думать прежде чем просто заполнять пропуски

Как заполнять пропуски?

- средним, медианой, другим значением
- "-1"
- "999"
- 'None'
- обучить модель предсказывать пропуск на основе других фичей (осторожно)

##### sklearn 

заполнение пропусков

https://scikit-learn.org/stable/modules/impute.html

In [ ]:
import sklearn
from sklearn.preprocessing import Imputer

In [ ]:
imp = Imputer(missing_values=np.nan, strategy='mean')

In [ ]:
df = pd.DataFrame({'площадь': [80, np.nan, 83, 74],
              'цена':[200, 250, 180, np.nan], 
              }) #'цена_кв_м':[200 / 80, 250 / 70, np.nan, np.nan]

In [ ]:
df

In [ ]:
df['площадь'] = imp.fit_transform(df['площадь'].values.reshape(-1, 1))
df['цена'] = imp.fit_transform(df['цена'].values.reshape(-1, 1))

In [ ]:
df

Пропуски в категориальных признаках
- создание отдельной категории ("нет значения")
- игнорирование пропусков (например, в dummy-кодировке сопоставить им нулевую строку, т.е. соответствующие объекты не
принадлежат ни одной категории)
- обработать пропуск ()

________-

# Выбросы

In [ ]:
# выбросы по квантилям
drop_rows = np.logical_or((tmp_x < tmp_x.quantile(q = 0.01)).any(axis=1), 
                          (tmp_x > tmp_x.quantile(q = 0.99)).any(axis=1))

winsorize

https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.mstats.winsorize.html

______

# Бинаризация

переход от вещественного признака к порядковому за счёт кодирования интервалов одним значением

как?

- диапазоны с одинаковой длинной
- диапазоны с одинаковым количеством точек
- кластеризация (нормер кластера или расстояние до центра кластера)
- руками (то есть экспертно)

In [ ]:
pd.cut(df['stand'] , 2)

In [ ]:
pd.qcut(df['stand'] , 3) # квантильная

как кодируем бинаризованные значения?
- наименьшим значением
- средним
- номером бина
- средним значением какого либо другого признака для объектов в одном бине

##### Sklearn

бинаризация непрерывных признаков в склерне.

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_classification.html

_____

# Нормализация

Для большинства алгоритмов машинного обучения необходимо, чтобы все признаки были вещественными и «в одной шкале».

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
w = np.array([1, 1, 1, 1, 1])
X = [1,2,2.4, 0.54, 5]

In [ ]:
from scipy.stats import rankdata

In [ ]:
plt.plot(X, w, 'ob', label='original')

X1 = X - np.mean(X)
X1 = X1 / np.std(X)

plt.plot(X1, w + 1, 'oy', label='z_score') # стандартизация


X1 = X - np.min(X)
X1 = X1 / (np.max(X) - np.min(X))


plt.plot(X1, w + 3, 'or', label='min_max scale')

RNK = rankdata(X, method='average')
plt.plot(RNK / np.max(RNK), w + 4, 'ok', label='rank')

plt.legend()

> можно делать нормировки в пределах группы, если она имеет смысл - смотрите данные. иногда это помогает. 


In [ ]:
df = pd.DataFrame({'sex': [0, 1, 1, 1, 0, 0],
                   'sum':[1000, 20000, 1500, 5000, 800, 50],
              })

df

In [ ]:
z_score = lambda x: (x - x.mean()) / x.std()
df['stand'] = df.groupby('sex').transform(z_score)

In [ ]:
df

Вещественные признаки

- деформация (функция над признаком)
- нормировка (специальный вид деформации) 
- новые признаки (функции над несколькими, например сумма, полиномы, и тд) 
- дискретизация (binning)

- делать деформацию, если она делает похожим распределение по признаку на нормальное (логарифм, корень, что нибудь специфичное)
- нормировка - приведение признаков в одну шкалу - для k-nn, SVM, logreg

# Дубликаты

Обычно они нам мешают, но не всегда. Бывает что просто иногда недостаточно данных. Тогда важно сохранить распределение классов.

In [ ]:
df = pd.DataFrame({'sex': [0, 1, 1, 1, 0, 0, 0],
                   'sum':[1000, 20000, 1500, 5000, 800, 50, 50],
              })

df

In [ ]:
df.duplicated(subset=['sex', 'sum'], keep='first')

# Новые признаки

Конкатенация категориальных

In [ ]:
df = pd.DataFrame({'sex': ['m', 'm', 'w', 'm', 'w'],
                   'sum':[2100, 14000, 20000, 1500, 2100],
              })

df

In [ ]:
df['new'] = df['sex'] + ' + ' + df['sum'].astype(str)

In [ ]:
df

##### Counter encoding

In [ ]:
_counter = {}

for i in df['sex'].values:
    _counter[i] = _counter.get(i,0) + 1

In [ ]:
_counter

> какие проблемы?

- коллизии 
- новые категории


Обычно мелкие категории объединяют в одну 

##### One hot

In [ ]:
df = pd.DataFrame({'sex': ['m', 'm', 'w', 'm', 'w', np.nan],
                   'sum':[2100, 14000, 20000, 1500, 2100, 0],
              })

df

In [ ]:
pd.get_dummies(df.sex)

In [ ]:
pd.get_dummies(df.sex, dummy_na=True,)

In [ ]:
pd.get_dummies(df.sex, drop_first=True)

_____

In [ ]:
data_train.start_time = pd.to_datetime(data_train.start_time)

Характеристика момента времени

- час, минута, секунда (=0)
- время суток
- день, день недели, день года
- неделя, месяц
- время года, год
- праздник / выходной / особый день (первый понедельник месяца,
начало Олимпиады)

для циклических  cos и sin

In [ ]:
def make_harmonic_features(value, period=24):
    value = (value / period) * 2 * np.pi
    return np.cos(value), np.sin(value)

from scipy.spatial import distance

distance.euclidean(make_harmonic_features(23), make_harmonic_features(1)), \
distance.euclidean(make_harmonic_features(1), make_harmonic_features(2)), \
distance.euclidean(make_harmonic_features(9), make_harmonic_features(15)), \
distance.euclidean(make_harmonic_features(9), make_harmonic_features(21))

Использовать пары признаков, типа разница времён
- близость к дедлайну
- в один ли день недели/год и т.п.
- сколько осталось до конца след дня (помните задачу авито?)
- 

Использовать время для других признаков 
- например чтобы отсортировать по времени и использовать весовые схемы (кнн какой нибудь)

- валидация!!!!!!!!!!

# Агрегирование

In [ ]:
df = pd.DataFrame({'p1': [10, 14, 24, 18],
                   'p2': [12, 18, 14, 16],
                   'p3': [8, 24, 22, 11],
                   'p4': [10, 15, 43, 18],
              })

In [ ]:
df['mean'] = df[['p1', 'p2', 'p3', 'p4']].mean(axis=1)
df['std'] = df[['p1', 'p2', 'p3', 'p4']].std(axis=1)
df['min'] = df[['p1', 'p2', 'p3', 'p4']].min(axis=1)
df['max'] = df[['p1', 'p2', 'p3', 'p4']].max(axis=1)
df['median'] = df[['p1', 'p2', 'p3', 'p4']].median(axis=1)

In [ ]:
df

In [ ]:
# иногда бывает важно посмотреть на расстояние от объекта до максимального, минимального ....

df['p2'] - df[['p1', 'p2', 'p3', 'p4']].max(axis=1)

##### Признаки -  агрегирование по разным источникам

In [ ]:
df = pd.DataFrame({'id': [1, 2, 3],
                   'sex': [0, 1, 1],
                   'age': [24, 54, 28],
                   'sum':[1000, 20000, 1500],
              })

appl_df = pd.DataFrame({'id': [1, 1, 1, 2, 2, 3, 3],
                   'date': [24, 25, 26, 14, 15, 16, 17],
                   'sum': [1000, 2000, 2000, 1000, 2500, 15000, 1800],
                   'просрочка':[0, 0, 1, 0, 0, 1, 1],
              })

In [ ]:
df

In [ ]:
appl_df

In [ ]:
appl_df.groupby('id')['просрочка'].mean().reset_index()

In [ ]:
appl_df.groupby(['id', 'sum'])['просрочка'].mean().reset_index()

> std, min, max, взвешиваем с весами, среднее + взвешенные с весами и тд